<h1>Data Gathering - Garda division and Green space Count </h1>

In this notebook , we get a count of the number of public pitches and green spaces in each garda subdistrict within Dublin.

- Although we know that our target area of The Liberties, Dublin 8 has insufficient green space we did not have an actually count of public green space or public pitches available in that area
- The availability of public green spaces and pitches is available from some county and city councils - this is useful for use but as we were dealing with smaller areas - Garda Subdivisions - we needed to break the county/city council count of green space availability down into a count for these smaller areas
- Furthermore, not all city/county councils provide the availability of green space in a format that we can readily use
- the following data was available from each of the county/city councils listed:
    - Fingal County Council - csv of public pitches
    - Dublin City Council - a map with markers of public pitches and parks
         - the positions of the markers were obtained from the network panal and saved to json files
    - Dun Laoghaire County Council -  csv's - public parks and pitches
    
- Galway and Roscommon green counts were also avaailable and taken as additional data, incase it was required at a later stage in the analysis
    - Galway City Council - csv's - public pitches and green spaces
    - Roscommon County Council - csv of public parks
       

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, MultiPoint, MultiPolygon
import geopandas as Gpd
from pyproj import Proj, transform
import json
%matplotlib inline

# "Census2011_Garda_SubDistricts_Nov2013.shp" is a shape file that contains the polygon coordinates of each garda subdivision in Ireland
# NOTE: This file was too large to submit on moodle 
# Link to files on google drive: https://drive.google.com/folderview?id=0ByQa-XM7CHA7d2d1Tk15MURhZjg&usp=sharing
# or it can be downloaded from: http://census.cso.ie/censusasp/saps/boundaries/ED_SA%20Disclaimer1.htm (Garda Sub Districts)
Census2011_Garda_SubDistricts_Nov2013.zip
shape = Gpd.GeoDataFrame.from_file("Census2011_Garda_SubDistricts_Nov2013.shp")

In [2]:
# Select all Dublin (DMR - Dublin Metropolitan Region) garda subdistricts from the geopandas dataframe containing all shapes for garda subdivision for all of Ireland 
dublin_area = shape.loc[shape['DIVISION'].str.contains("DMR")].reset_index(drop=True)

# find area of each division in km^2 (area in m^2/1000000)
dublin_area = {dublin_area['SUB_DIST'][i]:(dublin_area['geometry'][i].area/1000000) for i in range(0,len(dublin_area))}
dub_area = pd.DataFrame.from_dict(dublin_area,orient='index').reset_index()
dub_area.columns = ['GardaStation','Area']

# Drop the Dublin Subdivisions that we don't have sufficient green space data for
dub_area = dub_area[dub_area.GardaStation != 'Crumlin']
dub_area = dub_area[dub_area.GardaStation != 'Rathmines']
dub_area = dub_area[dub_area.GardaStation != 'Rathcoole']
dub_area = dub_area[dub_area.GardaStation != 'Terenure']
dub_area = dub_area[dub_area.GardaStation != 'Dundrum']
dub_area = dub_area[dub_area.GardaStation != 'Rathfarnham']
dub_area = dub_area[dub_area.GardaStation != 'Clondalkin']
dub_area = dub_area[dub_area.GardaStation != 'Ronanstown']
dub_area = dub_area[dub_area.GardaStation != 'Tallaght']
dub_area = dub_area.reset_index(drop=True)
print(dub_area)

         GardaStation        Area
0           Bridewell    3.005021
1               Lucan   27.031565
2          Balbriggan   60.000023
3       Pearse Street    3.182021
4           Irishtown   13.370166
5   Fitzgibbon Street    1.874715
6             Coolock   17.676889
7      Blanchardstown   45.287530
8            Skerries   30.605302
9                Lusk   54.564212
10         Cabinteely   18.411407
11           Shankill   20.719053
12           Clontarf   18.083254
13       Store Street    6.848610
14         Donnybrook    8.219549
15         Garristown   83.207734
16           Ballymun    7.025158
17            Airport    6.237030
18             Santry   15.133922
19      Dun Laoghaire   11.239514
20              Cabra   11.359962
21         Kilmainham    4.865736
22          Blackrock    8.965902
23      Sundrive Road    4.851760
24             Swords  105.245527
25        Ballyfermot    8.568579
26       Kevin Street    2.774478
27              Howth   14.487512
28           M

In [3]:
# convert the coordinates of the polygon shapes to geographical coordinates (latitude, longitude) so that the shapes can later be mapped onto an actual map 
shape = shape.to_crs(epsg=4326) 

In [4]:
# Extract shape information for each county of interest
shape_roscommon = shape.loc[shape['DIVISION'].str.contains("Roscommon")].reset_index(drop=True)
shape_galway = shape.loc[shape['DIVISION'].str.contains("Galway")].reset_index(drop=True)
shape_dublin = shape.loc[shape['DIVISION'].str.contains("DMR")].reset_index(drop=True)

<b>Data Sources </b>

The locations of public pitches and parks were obtained from a map detailing facilities in Dublin City Council. It can be found here: http://www.dublincity.ie/LocationPublisher/default.aspx?themename=Facilities&mapname=Sport.
Coordinates were obtained by going into the network panel and finding the map layer for the coordinates for playing pitches and then for parks and gardens. The coordinates were copied and saved in a json file which could then be opened here and the coordinates plotted on a map.

In [5]:
# load file with coordinates for playing pitches within Dublin City Council
with open('PlayingPitches.json') as data_file:    
    data_pitches = json.load(data_file)

# Coordinates are in the Spherical Mercator projection coordinate system(epsg:3857) so need to be transformed into geographical 
# coordinates (epsg:4326) in order to be able to put them onto the map here
inProj = Proj(init='epsg:3857')
outProj = Proj(init='epsg:4326')
LL_pitches = [0] * len(data_pitches['features'])

for i in range(0,len(data_pitches['features'])):
    latitude = data_pitches['features'][i]['geometry']['coordinates'][1]
    longitude = data_pitches['features'][i]['geometry']['coordinates'][0]
    lng, lat = transform(inProj, outProj, longitude, latitude)
    LL_pitches[i] = [lat,lng]
 
# lists containing longitudes and lattitudes of public pitches in Dublin
lons_pitches = [LL_pitches[i][0] for i in range(0, len(LL_pitches))]
lats_pitches = [LL_pitches[i][1] for i in range(0, len(LL_pitches))]

In [6]:
# the same process was follwed for the Parks as for the pitches within Dublin City Council
with open('ParksandOpenSpaces.json') as data_file:    
    data_parks = json.load(data_file)

inProj = Proj(init='epsg:3857')
outProj = Proj(init='epsg:4326')
LL_parks = [0] * len(data_parks['features'])

for i in range(0,len(data_parks['features'])):
    latitude = data_parks['features'][i]['geometry']['coordinates'][1]
    longitude = data_parks['features'][i]['geometry']['coordinates'][0]
    lng, lat = transform(inProj, outProj, longitude, latitude)
    LL_parks[i] = [lat,lng]

# lists containing longitudes and lattitudes of public parks in Dublin
lons_parks = [LL_parks[i][0] for i in range(0, len(LL_parks))]
lats_parks = [LL_parks[i][1] for i in range(0, len(LL_parks))]

In [7]:
# Pitches in Dun Laoghaire County Council were obtained from the data set on dublinked.ie
with open('dlrpitches.geojson') as data_file:    
    data_dlr = json.load(data_file)

# lists containing longitudes and lattitudes of public pitches in Dun Laoighaire
lat_dl = [(data_dlr['features'][i]['geometry']['coordinates'][0]) for i in range(0, len(data_dlr['features']))]
lon_dl = [(data_dlr['features'][i]['geometry']['coordinates'][1]) for i in range(0, len(data_dlr['features']))]

In [8]:
# Public Pitches in Fingal County Council
fcc_pitches = pd.read_csv('FCCPlayingPitches.csv')

# lists containing longitudes and latitudes of public pitches in Fingal County Council County
lat_fcc = [fcc_pitches['LAT'][i] for i in range(0,len(fcc_pitches['LAT']))]
lon_fcc = [fcc_pitches['LONG'][i] for i in range(0,len(fcc_pitches['LONG']))]

In [9]:
# Public parks in Galway City
gal_pitches = pd.read_csv('Parks_in_Galway_City.csv')

# lists containing longitudes and latitudes of public parks in Galway city
lat_gal = [gal_pitches['Lat'][i] for i in range(0,len(gal_pitches['Lat']))]
lon_gal = [gal_pitches['Long'][i] for i in range(0,len(gal_pitches['Long']))]

In [10]:
# Public parks in Roscommon
rcc_parks = pd.read_csv('RoscommonCommunityParks.csv')

# lists containing longitudes and latitudes of public parks in Roscommon
lon_rcc_park = [rcc_parks['\ufeffX'][i] for i in range(0,len(rcc_parks['WGS84Latitude']))]
lat_rcc_park = [rcc_parks['Y'][i] for i in range(0,len(rcc_parks['WGS84Latitude']))]

In [11]:
# Public parks in Roscommon
rcc_pitch = pd.read_csv('RoscommonPublicParks.csv')

# lists containing longitudes and latitudes of public parks in Roscommon
lon_rcc_pubpark = [rcc_pitch['\ufeffX'][i] for i in range(0,len(rcc_pitch['WGS84Latitude']))]
lat_rcc_pubpark = [rcc_pitch['Y'][i] for i in range(0,len(rcc_pitch['WGS84Latitude']))]

<b> Mapping Shape Files and park and pitch coordinates onto actual map </b>

The mplleaflet package was is used to map the shae files of the sub districts in dublin and all the coordinates of pitches and parks
This allows us to do a visual count of the pithces and parks within an area

Note: The html page of the following maps have been saved and submitted with the project - incase the mpleaflet package is not installed


In [12]:
import mplleaflet

# Dublin Map
ax = shape_dublin.plot()
ax.scatter(lats_pitches, lons_pitches, color="red")
ax.scatter(lats_parks, lons_parks, color="blue")
ax.scatter(lat_dl, lon_dl, color="green")
ax.scatter(lon_fcc, lat_fcc, color="cyan")
# display map here
#mplleaflet.display(fig=ax.figure)
# open map in a new window
mplleaflet.show(fig=ax.figure)

In [13]:
# Galway Map
ax = shape_galway.plot()
ax.scatter(lon_gal, lat_gal, color="black")
mplleaflet.show(fig=ax.figure)

In [14]:
# Roscommon Map
ax = shape_roscommon.plot()
ax.scatter(lon_rcc_park, lat_rcc_park, color="black")
ax.scatter(lon_rcc_pubpark, lat_rcc_pubpark, color="red")
mplleaflet.show(fig=ax.figure)

<b>Pitch and Park Count</b>

Counting the pitches and parks in a sub division is to check if a point is contained in a polygon. This is done below for each of the files containing coordinates of pitches and parks. This process is independent of the maps produced but in some cases the map was used to do a visual count were there was problems with checking the points in a polyon (which was the case for Galway City).
The maps also gave us a way to confirm the counts we obtained.

In [15]:
count = [0] * len(shape_dublin['geometry'])
# count of pitches in each subdivision in Dublin
for i in range(0, len(shape_dublin['geometry'])):
    poly = Polygon(shape_dublin['geometry'][i])
    # Dun Laoghaire pitches
    for j in range(0,len(lat_dl)):
        p = Point(lat_dl[j],lon_dl[j])
        if (poly.contains(p)):
            count[i] += 1
    # Dublin parks
    for j in range(0,len(LL_parks)):
        p = Point(LL_parks[j][1],LL_parks[j][0])
        if (poly.contains(p)):
            count[i] += 1
    # Dublin Pitches
    for j in range(0,len(LL_pitches)):
        p = Point(LL_pitches[j][1],LL_pitches[j][0])
        if (poly.contains(p)):
            count[i] += 1
    # Fingal Pitches
    for j in range(0,len(LL_pitches)):
        p = Point(lon_fcc[j],lat_fcc[j])
        if (poly.contains(p)):
            count[i] += 1
    
# make a dictionary of the number of pitches by sub division
dublin_dict = {shape_dublin['SUB_DIST'][i]:count[i] for i in range(0, len(count))}

# This file cotains the parks in dun laoghaire - they do not have the positions of the parks included so we just counted from the file ourselves
dl_park_file = pd.read_csv("dlrtotalfinal.csv")
print(dl_park_file)
# counts from dlr_park file
dublin_dict['Cabinteely'] += 8
dublin_dict['Dun Laoghaire'] += 2

# transform the dictionary into a dataframe
dublin = pd.DataFrame.from_dict(dublin_dict,orient='index').reset_index()
dublin.columns = ['GardaStation','Green']

# Drop the subdivisions in Dublin not being used
dublin = dublin[dublin.GardaStation != 'Crumlin']
dublin = dublin[dublin.GardaStation != 'Rathmines']
dublin = dublin[dublin.GardaStation != 'Rathcoole']
dublin = dublin[dublin.GardaStation != 'Terenure']
dublin = dublin[dublin.GardaStation != 'Dundrum']
dublin = dublin[dublin.GardaStation != 'Rathfarnham']
dublin = dublin[dublin.GardaStation != 'Clondalkin']
dublin = dublin[dublin.GardaStation != 'Ronanstown']
dublin = dublin[dublin.GardaStation != 'Tallaght']
dublin = dublin.reset_index(drop=True)

    Unnamed: 0                                         MAINTENANC DEPARTMENT
0            1  Marlfield: Open space at  Marlfield (gable of ...      Parks
1           38  Cabinteely Way (Gable 9 and rear 10 Monaloe Pa...      Parks
2          290                                  Cabinteely Avenue      Parks
3          291                                Cabinteely Crescent      Parks
4          483  Marlfield: Open space at  Marlfield (gable of ...      Parks
5          520  Cabinteely Way (Gable 9 and rear 10 Monaloe Pa...      Parks
6          772                                  Cabinteely Avenue      Parks
7          773                                Cabinteely Crescent      Parks
8          314                          Moran Park, Dun Laoghaire      Parks
9          315                       Beach Gardens, Dun Laoghaire      Parks
10         796                          Moran Park, Dun Laoghaire      Parks
11         797                       Beach Gardens, Dun Laoghaire      Parks

In [16]:
# count = [0] * len(shape_galway['geometry'])
# # count of pitches in each subdivision in Dun Laoghaire
# for i in range(0, len(shape_galway['geometry'])):
#     poly = Polygon(shape_galway['geometry'][i])
#     for j in range(0,len(lat_gal)):
#         p = Point(lon_gal[j],lat_gal[j])
#         if (poly.contains(p)):
#             count[i] += 1
            
# make a dictionary of the number of pitches by sub division
# galway_p = {shape_galway['SUB_DIST'][i]:count[i] for i in range(0, len(count))}

# galway = pd.DataFrame.from_dict(galway_p,orient='index')
# galway = galway.loc['Salthill'].append(dl_pitch.loc['Galway'])

# This count was obtained by looking at the map produced with Galway city garda sub division shapes and pitches
galway_dict = {'Galway': 19 , 'Salthill': 9}
galway = pd.DataFrame.from_dict(galway_dict,orient='index')
print(galway)

           0
Salthill   9
Galway    19


In [17]:
# The same process was followed here as for getting a count for Dublin pitches and parks
count = [0] * len(shape_roscommon['geometry'])

# count of pitches in each subdivision in Roscommon
for i in range(0, len(shape_roscommon['geometry'])):
    poly = Polygon(shape_roscommon['geometry'][i])
    for j in range(0,len(lat_rcc_park)):
        p = Point(lon_rcc_park[j],lat_rcc_park[j])
        if (poly.contains(p)):
            count[i] += 1
    for k in range(0,len(lat_rcc_pubpark)):
        p = Point(lon_rcc_pubpark[k],lat_rcc_pubpark[k])
        if (poly.contains(p)):
            count[i] += 1
            
# make a dictionary of the number of pitches by sub division
roscommon_dict = {shape_roscommon['SUB_DIST'][i]:count[i] for i in range(0, len(count))}

# Get rid of subdivisions in Roscommon that we dont have a grren space count for (value == 0)
roscommon_dict = {k:v for k,v in roscommon_dict.items() if v != 0}
roscommon = pd.DataFrame.from_dict(roscommon_dict,orient='index')
print(roscommon)

                 0
Athleague        1
Roscommon        3
Castlerea        1
Keadue           2
Boyle            3
Ballaghaderreen  1


In [18]:
# concatenate dataframes countaining green space count for subdivisions of each county
frames = [galway, dublin, roscommon]
result = pd.concat(frames)
result.to_csv("GreenSpaceCount.csv")

/Users/JoanMcCarthy/anaconda/envs/comp47350/lib/python3.5/site-packages/pandas/indexes/api.py:71: RuntimeWarning: unorderable types: str() > int(), sort order is undefined for incomparable objects
  result = result.union(other)


In [19]:
# merge the dataframes containing the area of each subdivision in Dublin (to be used later to find the population density) 
# and the green space count for each sub division 
res = pd.merge(left=dublin, right=dub_area, how='left', left_on='GardaStation', right_on='GardaStation')
res.to_csv("DublinGreenSpace&Area.csv")